# Decorators
- If you have gotten this far in your python career congrats nerd.
- Here is [PEP318](https://www.python.org/dev/peps/pep-0318), it's all about decorators in python. Go a head try to read it. I have tried several times and it sucks either because it is confusing as shit or I'm stupid. IDK you decide...
- Some of the code below comes from this [pretty good stackoverflow example](http://stackoverflow.com/questions/114214/class-method-differences-in-python-bound-unbound-and-static/114267#114267)

In [78]:
# Import pandas and numpy for illustration purposes.
import pandas as pd
import numpy as np

class TestData(object):
    
    def selfDataFrame(self):
        return pd.DataFrame(np.random.rand(10), columns=['A'])
    @classmethod
    def classDataFrame(cls):
        return pd.DataFrame(np.random.rand(10), columns=['A'])
    @staticmethod
    def staticDataFrame():
        return pd.DataFrame(np.random.rand(10), columns=['A'])

# myTest = TestData()
# TestData.selfDataFrame(object)
# TestData.classDataFrame()
try:
    TestData.classDataFrame(object)
except TypeError as err:
    print("Awww dang... @classmethod can'")
# TestData.staticDataFrame()

Awww dang... @classmethod can'


In [36]:
class Test(object):
    @classmethod
    def method_one(cls):
        print("Called method_one")
    @staticmethod
    def method_two():
        print("Called method_two")

# a_test = Test()
# a_test.method_one()
# a_test.method_two()
Test.method_two()

Called method_two


In [ ]:
>>> a_test = Test()
>>> a_test.method_one()
Called method_one
>>> a_test.method_two()
Called method_two
>>> Test.method_two()
Called method_two